## Machine learning

Iniziamo con la prima task, ovvero prevedere l'attività di twitter a livello provinciale, iniziamo assumendo una discretizzazione del tempo di 30 minuti in 30 minuti. Questo è dovuto al fatto che, grazie all'EDA, abbiamo osservato dei picchi di traffico specialmente nelle ore serali (da qui la necessità di avere una risoluzione temporale abbastanza fine) ma allo stesso tempo avendo 27k records in 62 giorni non volevamo avere dei binning con troppa poca popolazione. Da qui la nostra scelta.


In [66]:
from scipy import *
from numpy import *
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import numpy
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from functions import *
import seaborn as sns

# funzioni di sk-learn
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.metrics import matthews_corrcoef, accuracy_score, confusion_matrix, plot_confusion_matrix
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split


#custom lib
import make_dataset as m_d

ModuleNotFoundError: No module named 'pandas_profiling'

In [24]:
# ML regressors and classificators
from sklearn.linear_model import LogisticRegression

In [72]:
# devo prima unire i due dataset di line set -> nov e poi nov -> dicembre

#Problema: il primo dato vien preso come dizionario
#Converto colonne
temp = m_d.safe_import('SET-2')
temp=temp.rename(columns={'DG1000420': 'LINESET', "2013-12-01 00:00": 'Timestamp', "36.719997" : 'Value Amp'})
#Riaggiungo il primo dato
electro=appforth(temp,['DG1000420','2013-12-01 00:00',36.719997])

temp1 = m_d.safe_import('SET-2')
temp1=temp1.rename(columns={'DG1000420': 'LINESET', "2013-11-01 00:00": 'Timestamp', "37.439999" : 'Value Amp'})
#Riaggiungo il primo dato
electro_1=appforth(temp1,['DG1000420','2013-12-01 00:00',36.719997])

electro.append(electro_1)

lines = m_d.safe_import('SET-lines')
electro = electro.merge(right=lines, how='outer')


#Ci sono incompatibilità tra i record salvati quindi li droppo cattivo
electro = electro.rename(columns={'SQUAREID': 'cellId'})
electro = electro.dropna()


Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['SQUAREID', 'LINESET', 'NR_UBICAZIONI'], dtype='object')


In [73]:
electro

,LINESET,Timestamp,Value Amp,cellId,NR_UBICAZIONI
0,DG1000420,2013-12-01 00:00,36.719997,4037,2
1,DG1000420,2013-12-01 00:00,36.719997,4154,13
2,DG1000420,2013-12-01 00:00,36.719997,4155,20
3,DG1000420,2013-12-01 00:00,36.719997,4156,2
4,DG1000420,2013-12-01 00:00,36.719997,4269,1
...,...,...,...,...,...
11298379,DG1056626,2013-12-31 23:50,75.775002,8502,1
11298380,DG1056626,2013-12-31 23:50,75.775002,8503,4
11298381,DG1056626,2013-12-31 23:50,75.775002,8621,1
11298382,DG1056626,2013-12-31 23:50,75.775002,8972,25


In [19]:

### Per usare multi cartelle
#import sys
#sys.path.append('./../src')
#print(sys.path)

#data_path = Path('./data/raw')   #Per Cookiecutter
data_path = Path('./data/raw')
# subdivide the time series
def date_divider(df):
    """
    This Fuction extracts the data from an undivided data format YYYY-MM-DDThh-mm-ss
    Where T stands for Time and has no further meaning
    """
    column_names = ["month", "day", "hours", "temperature", "rain"]
    out = pd.DataFrame(columns=column_names)
    N = len(df["created"])  # Numero tweets creati in un giorno
    for i in range(0, N):
        # Inizializzo ogni riga o crasha
        out.loc[i] = "NaN"
        # Tempo
        # Tw_final.loc[i]["year"]=int(tweets.loc[i]["created"][0:4])  #Chemmifrega dell'anno, tutti uguali
        out.loc[i]["month"] = int(df.loc[i]["created"][5:7])
        out.loc[i]["day"] = int(df.loc[i]["created"][8:10])
        out.loc[i]["hours"] = int(df.loc[i]["created"][11:13]) + 0.5 * (
            int(df.loc[i]["created"][14:16]) >= 30)
    return out
"""
#Temperatura, per ricavare questa usiamo la stazione più vicina al tweet (sfortunamente il Trentino non ne ha tante)
dmin=10000000000
for st in weather.groupby(station):
    d=tweets["geometry"].distance(st[geometry])
gdf['distance'] = gdf['centroid'].distance(queens_point)
Tw_final.loc[i]["temperature"]=int(tweets.loc[i]["created"][5:7])
"""


"""
    NB:
    Support class to divide the hours dataset
    """
def Access_counter(df):
    """
    Please note that the data must be preprocessed by "date_divider" in order to work
    This function DOES NOT work with raw data
    This function assign a label to the minutes of the raw data in order to match them to the time resolution of the temperature aquisition.
    The result of this programme is a dictionair containing all the days!!
    """
    # I create a Support dictionary in order to get all the accesses subdivided by day
    dict = {}
    # Lunghezza is the number of elements
    lunghezza = 0
    k = 0
    mesi = df["month"].unique()
    for mese in mesi:
        temp = df[df["month"] == mese]
        lunghezza = len(temp['day'].unique())
        for j in range(lunghezza):
            data = temp[temp["day"] == j]
            bins = numpy.linspace(0, 24, 49)
            dict[k], edges = numpy.histogram(data['hours'], bins)
            k = k+1
    return dict

In [11]:
################### Importo le cose che servono, vanno cancellate in fase CC #################
df = pd.read_csv("twitter_final.csv")
df 

,month,day,hours,temperature,rain,municipal,geometry
0,11,1,0.0,13.2,0.0,Trento,POINT (11.13 46.07)
1,11,1,0.0,8.4,0.0,Pinzolo,POINT (10.83 46.23)
2,11,1,0.5,8.8,0.0,Cavalese,POINT (11.46 46.29)
3,11,1,0.5,13.2,0.0,Rovereto,POINT (11.04 45.89)
4,11,1,0.5,11.8,0.0,San Michele all'Adige,POINT (11.12 46.2)
...,...,...,...,...,...,...,...
27936,12,31,0.0,-7.7,0.0,Vigo di Fassa,POINT (11.68 46.42)
27937,12,29,13.0,3.4,0.0,Predazzo,POINT (11.6 46.31)
27938,11,19,0.5,5.0,0.0,Sant'Orsola Terme,POINT (11.3 46.11)
27939,11,23,9.5,0.7,0.0,Sant'Orsola Terme,POINT (11.3 46.11)


In [76]:
# grid di riferimento, serve un ponte tra l'ID della cella e la posizione geografica
grid=m_d.safe_import("grid")

grid.merge(right=electro, how = 'inner', on = 'cellId')
#grid.merge(right=electro, how = 'inner', on = 'geometry')

#df = df.merge(right=electro, how = 'inner', on = )

Index(['cellId', 'geometry'], dtype='object')


,cellId,geometry,LINESET,Timestamp,Value Amp,NR_UBICAZIONI
0,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:00,34.439999,4
1,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:10,34.799999,4
2,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:20,35.639999,4
3,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:30,32.520000,4
4,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:40,33.119999,4
...,...,...,...,...,...,...
11298379,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:10,12.825000,3
11298380,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:20,12.225000,3
11298381,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:30,12.100000,3
11298382,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:40,11.475000,3


In [81]:
# creo le il target, ovvero la divisione per fascia oraria di quanto ci è stato dato
counts_x_day = Access_counter(df)
print(counts_x_day[12])

[ 5  3  6  0  0  0  0  3  7  0  0  0  0 13 27  2  0  7  2  2 12  2  6  1
  3  0  2  0  4  2 23 21  7  4  5 14 35 19 11  2  4  6 11 16  8  4  3  1]


# Prepare the data for the ML dataset
Voglio preparare i dati in modo che siano della forma

X = ((Parametri giorno i-2),(parametri giorno i-1))

y = #tweets giorno i

In [111]:
# mi calcolo le somme dei tweets delle due distinzioni temporali che voglio assumere
# Queste funzioni risulteranno essere sia il target (delle 59 giornate che cercheremo di predire)
# sia il paramentro delle giornate successive alla loro
N_giorni =len(counts_x_day)
sum_mattina = np.zeros(N_giorni)
sum_sera = np.zeros(N_giorni )
for i in range(N_giorni):
    temp = counts_x_day[i]
    sum_mattina[i] = temp[linspace(15,37,dtype ='int')].sum() # 8->19
    sum_sera[i] = temp[linspace(38,47,dtype ='int')].sum()    # 19->24


Inizio a lavorare sulle temperature, dato che ho le temperature prese ogni 30 minuti decido di mediare sui due periodi SERA e MATTINA, in modo da aver un dato valutativo medio decente

In [121]:
# Adesso voglio  costruirmi un vettore che contenga le
dfTemp = df.drop(columns = ['rain','municipal','geometry'])
dfTemp_mattina = dfTemp[7.5 < dfTemp['hours'] < 19]
dfTemp_sera = dfTemp[19 < dfTemp['hours']]
#group_df = df.groupby(['month','day'])
#for i in range(N_giorni):
#    T = group_df['temperature'].
#print(T)
#T_mattina = np.zeros(N_giorni)
#T_sera = np.zeros(N_giorni)
#for i in range(N_giorni):
#    temp =     # scelta infelice di chiamare 
#    T_mattina[i] = temp[linspace(15,37,dtype ='int')].sum() # 8->19
#    T_sera[i] = temp[linspace(38,47,dtype ='int')].sum()    # 19->24
#

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
columns = ['#tweetsMAT-2','#tweetsMAT-1','#tweetsSER-2','#tweetsSER-1',  ]
ML_df = pd.DataFrame(columns =)


In [103]:
type(linspace(16,38, 23)[4])

numpy.float64

In [100]:
r = counts_x_day[16]
r[linspace(16,38, 23, dtype ='int')]

array([ 1,  3,  7,  6,  4,  3,  3,  9, 12,  2,  7,  8,  6, 12,  5, 13, 11,
        7,  7,  6,  5,  7, 11], dtype=int64)

## Logistic Regressor

{0: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int64),
 1: array([ 2,  7,  2,  3,  2,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  2,
         2,  3,  4,  4,  6,  6, 16,  2,  9, 11, 13,  8,  6, 10,  7,  2,  6,
        13, 15,  9, 11,  7,  7,  9, 14,  9, 19, 13,  7,  1,  6],
       dtype=int64),
 2: array([ 2,  6,  1,  1,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  1,  3,  0,
         2,  6,  3,  2,  3,  7,  9,  6,  6, 10, 12,  8, 11, 11,  3,  7,  6,
         4, 15, 11,  7,  2,  3,  4, 10,  4,  7,  2,  3,  5,  4],
       dtype=int64),
 3: array([ 8,  2,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  7,
         2,  5,  5,  4,  4,  4, 10,  6,  2,  8,  6,  7,  5,  3,  2,  8,  2,
         4,  5,  1, 15,  4,  3,  5, 10,  2,  4,  5, 11,  5, 25],
       dtype=int64),
 4: array([ 7,  2,  0,  0,  1,  0,  1,  0,  0,  0,  0,  1,  1,  9,  9,  5,  3,
         2, 